# NDAWN Interpolator

In [2]:
#import libraries 
import arcpy
import requests
import os
import zipfile
import io
import pandas as pd
from arcpy.sa import *

In [3]:
#set arcpy environment, this is where this script will store the imported data
proj_dir = r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02"

arcpy.env.workspace = proj_dir

#allow files to be over written
arcpy.env.overwriteOutput = True

#test
arcpy.env.workspace

'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt02'

## ETL for wrangling NDAWN station location and weather data for last 30 days

In [4]:
#request data from NDAWN API

#Set base url
ndawn_url = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&ttype=daily&quick_pick=30_d'
#ndawn_url = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&year=2022&ttype=daily&quick_pick=30_d&begin_date=2022-11-27&end_date=2022-11-27'

#request

get_data = requests.get(ndawn_url)

#open and decode contents of request

with open('AvgTemp_AllStations.csv', 'w') as AvgTempsAll:
    AvgTempsAll.write(get_data.content.decode('utf-8')) #decode with unicode 8 parameters
avg_temp_df = pd.read_csv('AvgTemp_AllStations.csv', skiprows = 5)
avg_temp_df = avg_temp_df.iloc[1:] #initializes df from row 2 to skip the messy labels in row index 0
avg_temp_df

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Min Temp,Min Temp Flag,Avg Temp,Avg Temp Flag
1,Ada,47.32119,-96.51406,910,2022.0,10.0,31.0,64.022,NaN,35.074,NaN,49.548,NaN
2,Ada,47.32119,-96.51406,910,2022.0,11.0,1.0,70.610,NaN,27.538,NaN,49.074,NaN
3,Ada,47.32119,-96.51406,910,2022.0,11.0,2.0,68.216,NaN,51.998,NaN,60.107,NaN
4,Ada,47.32119,-96.51406,910,2022.0,11.0,3.0,54.500,NaN,32.760,NaN,43.630,NaN
5,Ada,47.32119,-96.51406,910,2022.0,11.0,4.0,40.240,NaN,16.214,NaN,28.227,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,25.0,42.800,NaN,14.900,NaN,28.850,NaN
4227,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,26.0,38.952,NaN,22.595,NaN,30.774,NaN
4228,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,27.0,34.257,NaN,8.978,NaN,21.618,NaN
4229,Zeeland,46.01351,-99.68768,2070,2022.0,11.0,28.0,31.883,NaN,13.964,NaN,22.924,NaN


In [44]:
#save the new df to a csv
avg_temp_df.to_csv('StnLocMinMaxAvgTemp.csv')

## Map station locations and monthly average temp



In [56]:
# convert the csv to points, add to map
arcpy.management.XYTableToPoint("\\StnLocMinMaxAvgTemp.csv", "all_stn_loc","Longitude","Latitude") # optional params: ({z_field}, {coordinate_system})
#arcpy.management.XYTableToPoint(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\StnLocMinMaxAvgTemp.csv", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\StnLocMinMaxAvgTemp_XYTableToPoint", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt02\\all_stn_loc.shp'>

In [65]:
#Change symbology of the layer to display average temp at locations

# define project
project_path = r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.aprx"
p = arcpy.mp.ArcGISProject("CURRENT")

# specify map in project named "Map"
m = p.listMaps('Map')[0]

# specify layer in map to adjust named "StnLocMinMaxAvgTemp_XYTableToPoint"
lyr = m.listLayers('all_stn_loc')[0]

# access symbology of layer and change symbology to graduated colors
sym = lyr.symbology
sym.updateRenderer('GraduatedColorsRenderer')
sym.renderer.classificationField = 'Avg_Temp'
sym.classificationMethod = 'NaturalBreaks'
sym.renderer.breakCount = 5
colorRamp = p.listColorRamps("Plasma")[0]
sym.renderer.colorRamp = colorRamp

# save symbology back onto layer
lyr.symbology = sym

# save project
p.save()

In [6]:
# break up all stations df by station name and average values by station name
#turns out this wont work bc the temp data is not a numeric data type
#will attempt to fix this by cansting the temp columns as float type

#cast data type of column as float to calc stats
avg_temp_dfavg_temp_df['Max Temp'] = avg_temp_df['Max Temp'].astype(float)
avg_temp_dfavg_temp_df['Min Temp'] = avg_temp_df['Min Temp'].astype(float)
avg_temp_dfavg_temp_df['Avg Temp'] = avg_temp_df['Avg Temp'].astype(float)

avg_30d_temp = avg_temp_df.groupby(['Station Name']).mean()#.reset_index
print (avg_30d_temp) #reset_index())

NameError: name 'avg_temp_df' is not defined

In [101]:
stn_ada = avg_temp_df.loc[avg_temp_df['Station Name'] == 'Ada']
#stn_ada.describe(include = 'all')

#cast data type of column as float to calc stats
stn_ada['Max Temp'] = stn_ada['Max Temp'].astype(float)
stn_ada['Min Temp'] = stn_ada['Min Temp'].astype(float)
stn_ada['Avg Temp'] = stn_ada['Avg Temp'].astype(float)

#calc stats
stn_ada_30dMax = stn_ada['Max Temp'].mean()
stn_ada_30dMin = stn_ada['Min Temp'].mean()
stn_ada_30dAvg = stn_ada['Avg Temp'].mean()
print('30 day max temp:', stn_ada_30dMax)
print('30 day min temp:', stn_ada_30dMin)
print('30 day avg temp:', stn_ada_30dAvg)
stn_ada

[101]:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
[101]:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
[101]:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


30 day max temp: 37.142999999999994
30 day min temp: 20.608200000000004
30 day avg temp: 28.8758


,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Min Temp,Min Temp Flag,Avg Temp,Avg Temp Flag
1,Ada,47.32119,-96.51406,910,2022.0,10.0,29.0,60.584,NaN,30.619,NaN,45.602,NaN
2,Ada,47.32119,-96.51406,910,2022.0,10.0,30.0,58.784,NaN,22.091,NaN,40.438,NaN
3,Ada,47.32119,-96.51406,910,2022.0,10.0,31.0,64.022,NaN,35.074,NaN,49.548,NaN
4,Ada,47.32119,-96.51406,910,2022.0,11.0,1.0,70.610,NaN,27.538,NaN,49.074,NaN
5,Ada,47.32119,-96.51406,910,2022.0,11.0,2.0,68.216,NaN,51.998,NaN,60.107,NaN
6,Ada,47.32119,-96.51406,910,2022.0,11.0,3.0,54.500,NaN,32.760,NaN,43.630,NaN
7,Ada,47.32119,-96.51406,910,2022.0,11.0,4.0,40.240,NaN,16.214,NaN,28.227,NaN
8,Ada,47.32119,-96.51406,910,2022.0,11.0,5.0,44.958,NaN,15.710,NaN,30.334,NaN
9,Ada,47.32119,-96.51406,910,2022.0,11.0,6.0,42.814,NaN,24.031,NaN,33.423,NaN
10,Ada,47.32119,-96.51406,910,2022.0,11.0,7.0,35.942,NaN,15.674,NaN,25.808,NaN


## Interpolation for min and max temps 4 ways

### IDW


In [1]:
#max temp
out_raster = arcpy.sa.Idw("all_stn_loc", "Max_Temp", 0.0144790399999999, 2, "VARIABLE 12", None); out_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\Idw_all_stn_max")

#min temp
out_raster = arcpy.sa.Idw("all_stn_loc", "Min_Temp", 0.0144790399999999, 2, "VARIABLE 12", None); out_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\Idw_all_stn_min")

### Orinary Kriging

In [ ]:
#max temps
out_surface_raster = arcpy.sa.Kriging("all_stn_loc", "Max_Temp", "Spherical # # # #", 0.0144790399999999, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\Kriging_all_max")

#min temps
out_surface_raster = arcpy.sa.Kriging("all_stn_loc", "Min_Temp", "Spherical # # # #", 0.0144790399999999, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\Kriging_all_min")


### Empirical Bayesian kriging
#### an automated, randomized version of the kriging method based on multiple simulations

In [ ]:
# max temp
arcpy.ga.EmpiricalBayesianKriging("all_stn_loc", "Max_Temp", "ebk_semivariogram_max", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\ebk_all_max", 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

# min temp
arcpy.ga.EmpiricalBayesianKriging("all_stn_loc", "Min_Temp", "ebk_semivariogram_max", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\ebk_all_min", 0.0144790399999999, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

### Spline

In [ ]:
# max temps
out_raster = arcpy.sa.Spline("all_stn_loc", "Max_Temp", 0.0144790399999999, "REGULARIZED", 0.1, 12); out_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\Spline_all_max")

# min temps
out_raster = arcpy.sa.Spline("all_stn_loc", "Min_Temp", 0.0144790399999999, "REGULARIZED", 0.1, 12); out_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt02\L03pt02.gdb\Spline_all_min")